Método de cálculo para participaciones federales a alcaldías de Ciudad de México


In [1]:
# Limpieza del entorno de trabajo
rm(list=ls())

# Instalación y carga de paquetes necesarios ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, readxl, janitor, writexl, srvyr, httr)

Cargando paquete requerido: pacman



Tablas con claves de los municipios que conforman la Zona Metropolitana de la Ciudad de México

In [44]:
# Configuración de URLs y directorios temporales
url_datos <- 'https://www.gob.mx/cms/uploads/attachment/file/865680/Cuadros_MM2020.xlsx'
dir_temp <- 'temp'
archivo_temp <- file.path(dir_temp, 'datos_mm2020.xlsx')

#Crear directorio temporal si no existe
if (!dir.exists(dir_temp)) {
  dir.create(dir_temp)
}

#Descargar el archivo con manejo de errores
tryCatch({
  download.file(url_datos, archivo_temp, mode = 'wb')
  print('Archivo se descargó exitosamente')
}, error = function(e) {
  stop('Error al descargar: ', e$message)
})

#Leer la hoja 4 del archivo Excel
cve_zm <- tryCatch({
  read_xlsx(archivo_temp, sheet = 4) %>% 
  clean_names %>%
  filter(nombre_de_la_metropoli == 'Ciudad de México'                                                                 )
}, error = function(e) {
  stop('Error al leer el archivo Excel: ', e$message)
})

#Mostrar estructura den la tabla
names(cve_zm)


[1] "Archivo se descargó exitosamente"


[1] "clave_de_metropoli"                                                                        
 [2] "tipo_de_metropoli"                                                                         
 [3] "nombre_de_la_metropoli"                                                                    
 [4] "clave_de_la_entidad"                                                                       
 [5] "nombre_de_la_entidad"                                                                      
 [6] "clave_de_municipio"                                                                        
 [7] "clave_compuesta_del_municipio"                                                             
 [8] "nombre_del_municipio"                                                                      
 [9] "municipios_centrales_conurbacion_fisica"                                                   
[10] "municipios_centrales_integracion_funcional"                                                
[11] "municipios_centrales_localidad_o_conurbacion_de_200_mil_o_mas_habitantes_o_capital_estatal"
[12] "municipios_centrales_localidad_o_conurbacion_de_100_mil_o_mas_habitantes"                  
[13] "municipios_centrales_localidad_o_conurbacion_de_50_mil_o_mas_habitantes"                   
[14] "municipios_exteriores_integracion_funcional"                                               
[15] "municipios_exteriores_continuidad_geografica"

In [106]:
#Claves de alcaldia de trabajo
cve_cdmx <- cve_zm %>%
filter(clave_de_la_entidad == '09') %>% 
mutate(cve_ent = as.numeric(clave_de_la_entidad),
    cve_mun = as.numeric(clave_de_municipio)) %>%
select(cve_mun, nombre_del_municipio) 
#pull(cve_mun) #No correrlo si se quiere mantenerlo en forma de lista 

cve_cdmx

cve_mun,nombre_del_municipio
<dbl>,<chr>
2,Azcapotzalco
3,Coyoacán
4,Cuajimalpa de Morelos
5,Gustavo A. Madero
6,Iztacalco
7,Iztapalapa
8,La Magdalena Contreras
9,Milpa Alta
10,Álvaro Obregón


In [56]:
#Claves de municipios Edomex que viajan a la Ciudad
cve_edomex <- cve_zm %>%
filter(clave_de_la_entidad == '15') %>% 
mutate(cve_ent = as.numeric(clave_de_la_entidad),
    cve_mun = as.numeric(clave_de_municipio)) %>%
select(cve_ent, nombre_de_la_entidad, cve_mun, nombre_del_municipio) %>%
pull(cve_mun) #No correrlo si se quiere mantenerlo en forma de lista 


cve_edomex

[1]   2  11  13  20  22  23  24  25  28  29  30  31  33  35  37  38  39  44  46
[20]  53  57  58  59  60  69  70  75  81  83  84  89  91  92  93  95  99 100 103
[39] 104 108 109 120 121 122 125

In [58]:
#Claves de municipios Hidalgo que viajan a la Ciudad
cve_hidalgo <- cve_zm %>%
filter(clave_de_la_entidad == '13') %>% 
mutate(cve_ent = as.numeric(clave_de_la_entidad),
    cve_mun = as.numeric(clave_de_municipio)) %>%
select(cve_ent, nombre_de_la_entidad, cve_mun, nombre_del_municipio) %>%
pull(cve_mun) #No correrlo si se quiere mantenerlo en forma de lista 


cve_hidalgo

[1] 13 69

Cálculo población flotante interna CDMX

In [71]:
# Configuración de URLs y directorios para datos del censo
url_censo <- "https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/Censo2020_CA_cdmx_csv.zip"
dir_censo <- "datos_censo"
archivo_zip <- file.path(dir_censo, "censo_cdmx.zip")

# Crear directorio si no existe
if (!dir.exists(dir_censo)) {
  dir.create(dir_censo)
}

# Descargar y descomprimir archivo
tryCatch({
  # Descargar archivo ZIP
  download.file(url_censo, archivo_zip, mode = "wb")
  print("Archivo ZIP descargado exitosamente")
  
  # Descomprimir archivo
  unzip(archivo_zip, exdir = dir_censo)
  print("Archivo descomprimido exitosamente")
  
}, error = function(e) {
  stop("Error en la descarga o descompresión: ", e$message)
})

[1] "Archivo ZIP descargado exitosamente"
[1] "Archivo descomprimido exitosamente"


In [72]:
# Configuración de URLs y directorios para datos del censo
url_censo <- 'https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/Censo2020_CA_cdmx_csv.zip'
dir_censo <- 'datos_censo'
archivo_zip <- file.path(dir_censo, 'censo_cdmx.zip')

# Crear directorio si no existe
if (!dir.exists(dir_censo)) {
  dir.create(dir_censo)
}

# Descargar y descomprimir archivo
tryCatch({
  # Descargar archivo ZIP
  download.file(url_censo, archivo_zip, mode = 'wb')
  print('Archivo ZIP descargado exitosamente')
  
  # Descomprimir archivo
  unzip(archivo_zip, exdir = dir_censo)
  print('Archivo descomprimido exitosamente')
  
}, error = function(e) {
  stop('Error en la descarga o descompresión: ', e$message)
})

[1] "Archivo ZIP descargado exitosamente"
[1] "Archivo descomprimido exitosamente"


In [73]:
# Leer el archivo CSV del censo
tryCatch({
  # Listar archivos CSV en el directorio
  archivos_csv <- list.files(dir_censo, pattern = '\\.csv$', full.names = TRUE, ignore.case = TRUE)
  
  # Leer el archivo CSV
  pob_cdmx <- read.csv(archivos_csv[3], fileEncoding = 'UTF-8') %>%
  clean_names()
  
  str(pob_cdmx)
  
}, error = function(e) {
  stop('Error al leer el csv: ', e$message)
})

'data.frame':	276007 obs. of  91 variables:
 $ ent               : int  9 9 9 9 9 9 9 9 9 9 ...
 $ mun               : int  2 2 2 2 2 2 2 2 2 2 ...
 $ loc50k            : int  1 1 1 1 1 1 1 1 1 1 ...
 $ id_viv            : num  9e+10 9e+10 9e+10 9e+10 9e+10 ...
 $ id_persona        : num  9e+15 9e+15 9e+15 9e+15 9e+15 ...
 $ cobertura         : int  2 2 2 2 2 2 2 2 2 2 ...
 $ estrato           : chr  "09-002-0001-00" "09-002-0001-00" "09-002-0001-00" "09-002-0001-00" ...
 $ upm               : int  1 1 1 1 1 1 1 1 1 1 ...
 $ factor            : int  27 27 27 27 27 27 27 27 27 27 ...
 $ clavivp           : int  1 1 1 1 1 1 1 1 1 1 ...
 $ numper            : int  1 2 3 4 5 6 8 7 2 4 ...
 $ sexo              : int  3 1 3 3 1 1 3 1 3 1 ...
 $ edad              : int  72 77 47 40 45 39 14 21 55 24 ...
 $ parentesco        : int  101 201 301 301 418 418 408 408 201 301 ...
 $ ident_madre       : int  97 97 1 1 97 96 3 3 97 2 ...
 $ ident_padre       : int  97 97 2 2 97 96 5 5 97 1 ...
 $ ent

In [74]:
str(pob_cdmx$ent_pais_trab)

 int [1:276007] 9 9 9 9 9 9 NA 9 9 9 ...


In [76]:
#Paso 1. Filtrar solo personas done ent_pais_trab = 9
cdmx <- pob_cdmx %>%
filter(ent_pais_trab == 9)

In [77]:
#Paso 2. Construir una tabla parecida a la dinamica en excel
tabla_cdmx <- cdmx %>%
group_by(mun_trab, mun) %>%
summarise(poblacion_total = sum(factor, na.rm = TRUE), .groups = 'drop') %>%
pivot_wider(
    names_from = mun,                   
    values_from = poblacion_total,      
    values_fill = 0) %>%                     
mutate(
    mun_trab =as.character(mun_trab),
    total_gral = rowSums(across(where(is.numeric)), na.rm = TRUE))

tail(tabla_cdmx)

mun_trab,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,total_gral
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
13,379,3116,109,1012,848,4743,509,5133,1407,4116,4853,122131,971,631,215,391,150564
14,5969,20116,1954,15474,13106,43935,5913,1238,32526,7375,16235,6171,126304,14645,9313,7816,328090
15,18290,19167,2178,55928,21981,62613,5212,1628,23637,10524,16208,8078,34217,205827,19415,31034,535937
16,21827,12642,6575,30856,10889,26043,4670,949,27923,4755,8524,3464,22893,20551,142956,9661,355178
17,1465,3675,311,18171,9443,18746,588,609,2777,1947,2042,1442,4174,5930,2238,125520,199078
999,223,3742,160,496,2315,3779,145,78,198,375,1951,635,1262,414,680,371,16824


In [78]:
#Paso 3. Eliminar el municipio “999” (no especificado)
tabla_cdmx <- tabla_cdmx %>%
filter(mun_trab != 999)
tabla_cdmx

mun_trab,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,total_gral
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
2,115492,1287,529,20770,1956,6368,415,165,2495,1218,1245,790,1739,5247,4821,2929,167466
3,2599,171126,902,8702,7257,35294,8640,3524,11710,10200,33754,14346,11840,6729,3342,4303,334268
4,2251,2996,80799,4006,1476,4589,2145,146,13882,1070,4520,1239,4927,3500,4269,1351,133166
5,8286,1882,626,313507,3677,8048,533,484,2138,1533,1624,1106,2474,5569,2572,7702,361761
6,678,2331,138,4106,85795,23170,323,387,1360,2241,2119,945,1999,2130,676,3665,132063
7,1198,8351,280,6085,15351,488442,833,3111,3693,19873,4020,4935,4057,2957,1537,4235,568958
8,322,1949,233,535,665,1880,51882,258,4425,704,7423,911,1189,530,475,211,73592
9,49,33,143,115,41,339,53,46934,0,1708,144,1755,190,0,0,54,51558
10,4658,11648,4820,9647,5543,16663,15249,753,219658,3639,18063,4591,12104,6372,8847,3565,345820


In [79]:
#Paso 4. Eliminar la diagonal principal (personas que viven y trabajan en la misma alcaldía). O sea,. 
#poner en cero los valores donde mun_trab == mun

for (col in intersect(names(tabla_cdmx), as.character(tabla_cdmx$mun_trab))) {
  tabla_cdmx[[col]] <- ifelse(tabla_cdmx$mun_trab == as.numeric(col), 0, tabla_cdmx[[col]])
}

# Paso 5: Agregar totales
tabla_cdmx <- tabla_cdmx %>%
  mutate(Total_general = rowSums(across(where(is.numeric)), na.rm = TRUE))

total_general <- tabla_cdmx %>%
  summarise(across(where(is.numeric), sum, na.rm = TRUE)) %>%
  mutate(mun_trab = 'Total general')

tabla_cdmx <- bind_rows(tabla_cdmx, total_general)

# Resultado final
tabla_cdmx


Warning message:
"There was 1 warning in `summarise()`.
ℹ In argument: `across(where(is.numeric), sum, na.rm = TRUE)`.
Caused by warning:
! The `...` argument of `across()` is deprecated as of dplyr 1.1.0.
Supply arguments directly to `.fns` through an anonymous function instead.

  # Previously
  across(a:b, mean, na.rm = TRUE)

  # Now
  across(a:b, \(x) mean(x, na.rm = TRUE))"


mun_trab,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,total_gral,Total_general
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,0,1287,529,20770,1956,6368,415,165,2495,1218,1245,790,1739,5247,4821,2929,167466,219440
3,2599,0,902,8702,7257,35294,8640,3524,11710,10200,33754,14346,11840,6729,3342,4303,334268,497410
4,2251,2996,0,4006,1476,4589,2145,146,13882,1070,4520,1239,4927,3500,4269,1351,133166,185533
5,8286,1882,626,0,3677,8048,533,484,2138,1533,1624,1106,2474,5569,2572,7702,361761,410015
6,678,2331,138,4106,0,23170,323,387,1360,2241,2119,945,1999,2130,676,3665,132063,178331
7,1198,8351,280,6085,15351,0,833,3111,3693,19873,4020,4935,4057,2957,1537,4235,568958,649474
8,322,1949,233,535,665,1880,0,258,4425,704,7423,911,1189,530,475,211,73592,95302
9,49,33,143,115,41,339,53,0,0,1708,144,1755,190,0,0,54,51558,56182
10,4658,11648,4820,9647,5543,16663,15249,753,0,3639,18063,4591,12104,6372,8847,3565,345820,471982


Cálculo población flotante proveniente del EdoMex

In [85]:
# Configuración de URLs y directorios para datos del censo (Edomex)
url_censo <- 'https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/Censo2020_CA_mex_csv.zip'
dir_censo <- 'datos_censo_edomex'
archivo_zip <- file.path(dir_censo, 'censo_edomex.zip')

# Crear directorio si no existe
if (!dir.exists(dir_censo)) {
  dir.create(dir_censo)
}

# Descargar y descomprimir archivo
tryCatch({
  # Descargar archivo ZIP
  download.file(url_censo, archivo_zip, mode = 'wb')
  message('Archivo ZIP descargado exitosamente')
  
  # Descomprimir archivo
  unzip(archivo_zip, exdir = dir_censo)
  message('Archivo descomprimido exitosamente')
  
}, error = function(e) {
  stop('Error en la descarga o descompresión: ', e$message)
})

Archivo ZIP descargado exitosamente

Archivo descomprimido exitosamente



In [87]:
#Leer el archivo CSV del censo
tryCatch({
  # Listar archivos CSV en el directorio
  archivos_csv <- list.files(dir_censo, pattern = '\\.csv$', full.names = TRUE, ignore.case = TRUE)
  
  # Mostrar archivos encontrados
  message('Archivos encontrados:')
  print(archivos_csv)
  
  # Leer el archivo CSV (elige el que contiene los datos de personas)
  pob_edomex <- read.csv(archivos_csv[2], fileEncoding = 'UTF-8') %>%
  clean_names()
  
  # Ver estructura del dataframe
  str(pob_edomex)
  
}, error = function(e) {
  stop('Error al leer el CSV: ', e$message)
})

Archivos encontrados:



[1] "datos_censo_edomex/Migrantes15.CSV" "datos_censo_edomex/Personas15.CSV" 
[3] "datos_censo_edomex/Viviendas15.CSV"
'data.frame':	1228485 obs. of  91 variables:
 $ ent               : int  15 15 15 15 15 15 15 15 15 15 ...
 $ mun               : int  1 1 1 1 1 1 1 1 1 1 ...
 $ loc50k            : int  0 0 0 0 0 0 0 0 0 0 ...
 $ id_viv            : num  1.5e+11 1.5e+11 1.5e+11 1.5e+11 1.5e+11 ...
 $ id_persona        : num  1.5e+16 1.5e+16 1.5e+16 1.5e+16 1.5e+16 ...
 $ cobertura         : int  2 2 2 2 2 2 2 2 2 2 ...
 $ estrato           : chr  "15-001-0000-04" "15-001-0000-04" "15-001-0000-04" "15-001-0000-04" ...
 $ upm               : int  1 1 1 1 1 1 1 1 1 1 ...
 $ factor            : int  9 9 9 9 9 9 9 9 9 9 ...
 $ clavivp           : int  1 1 1 2 2 2 1 1 1 1 ...
 $ numper            : int  3 2 1 2 3 1 1 4 3 2 ...
 $ sexo              : int  3 1 3 3 3 3 1 3 1 3 ...
 $ edad              : int  18 22 50 16 74 44 42 1 12 41 ...
 $ parentesco        : int  301 301 101 301 401 101 1

In [92]:
#Paso 1. Filtrar personas que viven en Edomex y trabajan en la CDMX
edomex <- pob_edomex %>%
filter(mun %in% c(cve_edomex)) %>% #municipios de la zona metropolitana
filter(ent == 15, ent_pais_trab == 9) %>%
select(mun, mun_trab, factor)

head(edomex)

,mun,mun_trab,factor
,<int>,<int>,<int>
1,2,17,21
2,2,2,20
3,2,999,20
4,2,999,20
5,2,999,20
6,2,999,20


In [96]:
#Paso 2. Agrupar y sumar el factor 
flotante_edomex <- edomex %>%
group_by(mun, mun_trab) %>%
summarise(poblacion_total = sum(factor, na.rm = TRUE), .groups = 'drop') %>%
pivot_wider(
    names_from = mun_trab,        # municipio de trabajo (CDMX)
    values_from = poblacion_total,
    values_fill = 0) %>%
mutate(total_general = rowSums(across(where(is.numeric)), na.rm = TRUE))

tail(flotante_edomex)

mun,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,999,total_general
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
108,945,189,152,525,95,114,11,7,312,49,212,46,282,802,1023,217,3088,8177
109,2943,595,384,2431,319,422,54,0,944,153,255,136,1200,1995,3288,614,11291,27133
120,1253,489,176,2010,457,684,52,18,717,69,445,5,863,1375,1185,696,3488,14102
121,3397,937,991,2886,127,458,40,0,1792,98,569,82,1999,2409,5841,442,12495,34684
122,824,3685,434,1161,2846,12938,235,192,1925,5786,1486,959,3665,6901,3364,3752,19137,69412
125,59,25,8,101,11,14,3,1,26,4,14,3,43,118,80,21,511,1167


Cálculo población flotante proveniente de Hidalgo

In [98]:
# Configuración de URLs y directorios para datos del censo (Edomex)
url_censo <- 'https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/Censo2020_CA_hgo_csv.zip'
dir_censo <- 'datos_censo_hgo'
archivo_zip <- file.path(dir_censo, 'censo_hgo.zip')

# Crear directorio si no existe
if (!dir.exists(dir_censo)) {
  dir.create(dir_censo)
}

# Descargar y descomprimir archivo
tryCatch({
  # Descargar archivo ZIP
  download.file(url_censo, archivo_zip, mode = 'wb')
  message('Archivo ZIP descargado exitosamente')
  
  # Descomprimir archivo
  unzip(archivo_zip, exdir = dir_censo)
  message('Archivo descomprimido exitosamente')
  
}, error = function(e) {
  stop('Error en la descarga o descompresión: ', e$message)
})

Archivo ZIP descargado exitosamente

Archivo descomprimido exitosamente



In [99]:
#Leer el archivo CSV del censo
tryCatch({
  # Listar archivos CSV en el directorio
  archivos_csv <- list.files(dir_censo, pattern = '\\.csv$', full.names = TRUE, ignore.case = TRUE)
  
  # Mostrar archivos encontrados
  message('Archivos encontrados:')
  print(archivos_csv)
  
  # Leer el archivo CSV (elige el que contiene los datos de personas)
  pob_hgo <- read.csv(archivos_csv[2], fileEncoding = 'UTF-8') %>%
  clean_names()
  
  # Ver estructura del dataframe
  str(pob_hgo)
  
}, error = function(e) {
  stop('Error al leer el CSV: ', e$message)
})

Archivos encontrados:



[1] "datos_censo_hgo/Migrantes13.CSV" "datos_censo_hgo/Personas13.CSV" 
[3] "datos_censo_hgo/Viviendas13.CSV"
'data.frame':	503228 obs. of  91 variables:
 $ ent               : int  13 13 13 13 13 13 13 13 13 13 ...
 $ mun               : int  1 1 1 1 1 1 1 1 1 1 ...
 $ loc50k            : int  0 0 0 0 0 0 0 0 0 0 ...
 $ id_viv            : num  1.3e+11 1.3e+11 1.3e+11 1.3e+11 1.3e+11 ...
 $ id_persona        : num  1.3e+16 1.3e+16 1.3e+16 1.3e+16 1.3e+16 ...
 $ cobertura         : int  2 2 2 2 2 2 2 2 2 2 ...
 $ estrato           : chr  "13-001-0000-03" "13-001-0000-03" "13-001-0000-03" "13-001-0000-03" ...
 $ upm               : int  1 1 1 1 1 1 1 1 1 1 ...
 $ factor            : int  4 4 4 4 4 4 4 4 4 4 ...
 $ clavivp           : int  1 1 1 1 4 4 4 4 4 4 ...
 $ numper            : int  3 2 1 4 1 1 2 3 1 2 ...
 $ sexo              : int  3 3 1 3 1 1 3 3 3 3 ...
 $ edad              : int  9 27 31 5 31 20 17 1 29 9 ...
 $ parentesco        : int  301 201 101 301 101 101 201 301 101 30

In [101]:
#Paso 1. Filtrar personas que viven en Edomex y trabajan en la CDMX
hidalgo <- pob_hgo %>%
filter(mun %in% c(cve_hidalgo)) %>% #municipios de la zona metropolitana
filter(ent == 13, ent_pais_trab == 9) %>%
select(mun, mun_trab, factor)

head(hidalgo)

,mun,mun_trab,factor
,<int>,<int>,<int>
1,13,999,6
2,13,10,5
3,13,14,6
4,13,16,6
5,13,3,11
6,13,999,11


In [102]:
#Paso 2. Agrupar y sumar el factor 
flotante_hgo <- hidalgo %>%
group_by(mun, mun_trab) %>%
summarise(poblacion_total = sum(factor, na.rm = TRUE), .groups = 'drop') %>%
pivot_wider(
    names_from = mun_trab,        # municipio de trabajo (CDMX)
    values_from = poblacion_total,
    values_fill = 0) %>%
mutate(total_general = rowSums(across(where(is.numeric)), na.rm = TRUE))

tail(flotante_hgo)

mun,2,3,4,5,6,7,10,11,12,14,15,16,17,999,8,13,total_general
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
13,154,75,44,193,11,77,94,33,24,92,89,170,32,223,0,0,1324
69,532,164,114,1350,106,314,435,171,321,485,1483,657,668,3473,14,13,10369


In [109]:
flotante_edomex <- flotante_edomex %>%
  rename(mun_trab = mun)  # ajusta el nombre real si es distinto

flotante_hgo <- flotante_hgo %>%
  rename(mun_trab = mun)  # o renómbrala igual
